# Binned statistics of ML heating/cooling based on advection in the full field

# StrongCurrent case - binned stats of ML heat balance

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cmocean as cmo
import glob
from tqdm import tqdm
import sys
import calendar

In [2]:
x_array = np.arange(0, 1122*2500, 2500)
y_array = np.arange(20*2500, 430*2500, 2500)
x_rho, y_rho = np.meshgrid(x_array, y_array)

# Monthly mean temp_rate and eddy advection

# Change the case_name to recreate files
### '../binned_stats_data/' + case_name + '_temp_rate_total_adv_binned_statistics_2016.nc'
### '../binned_stats_data/' + case_name + '_temp_rate_eddy_adv_binned_statistics_2016.nc'

In [3]:
case_name = 'Strong'
years = '[2-3]'

In [4]:
heat_balance_files = glob.glob('../monthly_ml_heat_budget_large_mesoscale/'
                                + case_name + '/monthly_ml_heat_balance_2016-*_' + case_name + 'Eddies.nc')
heat_balance_files = sorted(heat_balance_files)

In [5]:
%%time
temp_rate_ml, eddy_adv_ml, temp_adv_ml = np.zeros([len(heat_balance_files), 410, 1122]), np.zeros([len(heat_balance_files), 410, 1122]), np.zeros([len(heat_balance_files), 410, 1122])
for num, temp_rate_ml_eddy_file in enumerate(heat_balance_files):
    temp_rate_ml_ds = xr.open_dataset(temp_rate_ml_eddy_file)
    temp_rate_ml[num] = temp_rate_ml_ds.temp_rate_ml.data
    eddy_adv_ml[num] = temp_rate_ml_ds.eddy_adv_ml.data
    temp_adv_ml[num] = temp_rate_ml_ds.temp_adv_ml.data

CPU times: user 187 ms, sys: 137 ms, total: 324 ms
Wall time: 2.37 s


In [6]:
MS_ds = xr.Dataset(data_vars={'eddy_adv_ml' : (('ocean_time','eta_rho', 'xi_rho'), eddy_adv_ml[:, 60:350, 60:1062]),
                            'temp_rate_ml' : (('ocean_time','eta_rho', 'xi_rho'), temp_rate_ml[:, 60:350, 60:1062]),
                            'x_rho' : (('eta_rho', 'xi_rho'), x_rho[60:350, 60:1062]),
                            'y_rho' : (('eta_rho', 'xi_rho'), y_rho[60:350, 60:1062])},
                coords={ 'eta_rho':   y_rho[60:350,0], 'xi_rho':    x_rho[0, 60:1062]})
MS_ds

<xarray.Dataset>
Dimensions:       (eta_rho: 290, ocean_time: 12, xi_rho: 1002)
Coordinates:
  * eta_rho       (eta_rho) int64 200000 202500 205000 ... 917500 920000 922500
  * xi_rho        (xi_rho) int64 150000 152500 155000 ... 2650000 2652500
Dimensions without coordinates: ocean_time
Data variables:
    eddy_adv_ml   (ocean_time, eta_rho, xi_rho) float64 -40.69 -42.53 ... -14.4
    temp_rate_ml  (ocean_time, eta_rho, xi_rho) float64 62.21 63.78 ... -40.56
    x_rho         (eta_rho, xi_rho) int64 150000 152500 ... 2650000 2652500
    y_rho         (eta_rho, xi_rho) int64 200000 200000 200000 ... 922500 922500

## coefficients for binned statistics

In [7]:
population_cutoff = 5000
eddy_adv_bins = np.arange(-2000, 2000, 100)

# Eddy adv and ML heating/cooling

In [8]:
meso_grouped_mean = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).mean()
meso_grouped_std = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).std()

# calculating bin population
meso_groups = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).groups
    
count = 0
meso_bin_popu = np.zeros([len(meso_groups.keys()), 2])
for key in meso_groups.keys():
    meso_bin_popu[count] = key.left, len(meso_groups[key])
    count += 1
    
# finding the bins 
meso_valid = np.where(meso_bin_popu[meso_bin_popu[:, 0].argsort(), 1] > population_cutoff)

In [9]:
meso_grouped_mean

<xarray.Dataset>
Dimensions:           (eddy_adv_ml_bins: 39)
Coordinates:
  * eddy_adv_ml_bins  (eddy_adv_ml_bins) object (-2000, -1900] ... (1800, 1900]
Data variables:
    eddy_adv_ml       (eddy_adv_ml_bins) float64 nan -1.814e+03 ... 1.837e+03
    temp_rate_ml      (eddy_adv_ml_bins) float64 nan 610.1 ... -1.955e+03
    x_rho             (eddy_adv_ml_bins) float64 nan 2.385e+06 ... 2.552e+06
    y_rho             (eddy_adv_ml_bins) float64 nan 8.9e+05 ... 9.15e+05

In [10]:
eddy_adv_ml_data, MS_SSTA_mean_data, MS_SSTA_std_data = meso_grouped_mean.eddy_adv_ml[meso_valid].data, meso_grouped_mean.temp_rate_ml[meso_valid].data, meso_grouped_std.temp_rate_ml[meso_valid].data
MS_bin_popu_data = meso_bin_popu[meso_bin_popu[:, 0].argsort(), 1][meso_valid]
#mean_adv_ml_data, LS_SSTA_mean_data, LS_SSTA_std_data, LS_bin_popu_data

In [11]:
eddy_stats_ds = xr.Dataset(data_vars={'eddy_adv_ml_bin_left' : (('temp_adv'), eddy_adv_ml_data),
                           'temp_rate_ml_mean' : (('temp_adv'), MS_SSTA_mean_data),
                           'temp_rate_ml_std' : (('temp_adv'), MS_SSTA_std_data),
                           'eddy_adv_ml_bin_population' : (('temp_adv'), MS_bin_popu_data)},
                coords={ 'eddy_adv': np.arange(len(MS_bin_popu_data))})
eddy_stats_ds

<xarray.Dataset>
Dimensions:                     (eddy_adv: 18, temp_adv: 18)
Coordinates:
  * eddy_adv                    (eddy_adv) int64 0 1 2 3 4 5 ... 13 14 15 16 17
Dimensions without coordinates: temp_adv
Data variables:
    eddy_adv_ml_bin_left        (temp_adv) float64 -1.147e+03 ... 544.6
    temp_rate_ml_mean           (temp_adv) float64 255.0 231.6 ... -335.7 -472.3
    temp_rate_ml_std            (temp_adv) float64 333.5 327.5 ... 196.5 211.3
    eddy_adv_ml_bin_population  (temp_adv) float64 5.183e+03 ... 7.306e+03

In [12]:
eddy_stats_ds.to_netcdf('./' + case_name + '_temp_rate_eddy_adv_binned_statistics_2016.nc')

# Total adv and ML heating/cooling

In [13]:
total_ds = xr.Dataset(data_vars={'temp_adv_ml' : (('ocean_time','eta_rho', 'xi_rho'), temp_adv_ml[:]),
                            'temp_rate_ml' : (('ocean_time','eta_rho', 'xi_rho'), temp_rate_ml[:]),
                            'x_rho' : (('eta_rho', 'xi_rho'), x_rho),
                            'y_rho' : (('eta_rho', 'xi_rho'), y_rho)},
                coords={ 'eta_rho':   y_rho[:,0], 'xi_rho':    x_rho[0, :]})
total_ds

<xarray.Dataset>
Dimensions:       (eta_rho: 410, ocean_time: 12, xi_rho: 1122)
Coordinates:
  * eta_rho       (eta_rho) int64 50000 52500 55000 ... 1067500 1070000 1072500
  * xi_rho        (xi_rho) int64 0 2500 5000 7500 ... 2797500 2800000 2802500
Dimensions without coordinates: ocean_time
Data variables:
    temp_adv_ml   (ocean_time, eta_rho, xi_rho) float64 1.675e+03 ... 436.8
    temp_rate_ml  (ocean_time, eta_rho, xi_rho) float64 17.22 46.59 ... 31.08
    x_rho         (eta_rho, xi_rho) int64 0 2500 5000 ... 2800000 2802500
    y_rho         (eta_rho, xi_rho) int64 50000 50000 50000 ... 1072500 1072500

In [14]:
total_adv_bins = np.arange(-800, 800, 25)
total_grouped_mean = total_ds.groupby_bins('temp_adv_ml', total_adv_bins, squeeze=False).mean()
total_grouped_std = total_ds.groupby_bins('temp_adv_ml', total_adv_bins, squeeze=False).std()
# calculating bin population
total_groups = total_ds.groupby_bins('temp_adv_ml', total_adv_bins, squeeze=False).groups   
count = 0
total_bin_popu = np.zeros([len(total_groups.keys()), 2])
for key in total_groups.keys():
    total_bin_popu[count] = key.left, len(total_groups[key])
    count += 1   
# finding the bins 
total_valid = np.where(total_bin_popu[total_bin_popu[:, 0].argsort(), 1] > population_cutoff)

In [15]:
total_grouped_mean

<xarray.Dataset>
Dimensions:           (temp_adv_ml_bins: 63)
Coordinates:
  * temp_adv_ml_bins  (temp_adv_ml_bins) object (-800, -775] ... (750, 775]
Data variables:
    temp_adv_ml       (temp_adv_ml_bins) float64 -787.6 -762.4 ... 737.6 762.1
    temp_rate_ml      (temp_adv_ml_bins) float64 -758.1 -744.4 ... 535.4 546.6
    x_rho             (temp_adv_ml_bins) float64 1.239e+06 ... 1.365e+06
    y_rho             (temp_adv_ml_bins) float64 8.243e+05 ... 7.966e+05

In [16]:
total_adv_ml_data, total_SSTA_mean_data, total_SSTA_std_data = total_grouped_mean.temp_adv_ml[total_valid].data, total_grouped_mean.temp_rate_ml[total_valid].data, total_grouped_std.temp_rate_ml[total_valid].data
total_bin_popu_data = total_bin_popu[total_bin_popu[:, 0].argsort(), 1][total_valid]

In [17]:
total_stats_ds = xr.Dataset(data_vars={'temp_adv_ml_bin_left' : (('temp_adv'), total_adv_ml_data),
                           'temp_rate_ml_mean' : (('temp_adv'), total_SSTA_mean_data),
                           'temp_rate_ml_std' : (('temp_adv'), total_SSTA_std_data),
                           'eddy_adv_ml_bin_population' : (('temp_adv'), total_bin_popu_data)},
                coords={ 'eddy_adv': np.arange(len(MS_bin_popu_data))})
total_stats_ds

<xarray.Dataset>
Dimensions:                     (eddy_adv: 18, temp_adv: 41)
Coordinates:
  * eddy_adv                    (eddy_adv) int64 0 1 2 3 4 5 ... 13 14 15 16 17
Dimensions without coordinates: temp_adv
Data variables:
    temp_adv_ml_bin_left        (temp_adv) float64 -562.2 -537.3 ... 412.1 437.1
    temp_rate_ml_mean           (temp_adv) float64 -569.9 -545.1 ... 303.0 332.2
    temp_rate_ml_std            (temp_adv) float64 113.5 112.4 ... 131.2 132.2
    eddy_adv_ml_bin_population  (temp_adv) float64 5.098e+03 ... 5.862e+03

In [18]:
total_stats_ds.to_netcdf('./' + case_name + '_temp_rate_total_adv_binned_statistics_2016.nc')

In [19]:
total_stats_ds

<xarray.Dataset>
Dimensions:                     (eddy_adv: 18, temp_adv: 41)
Coordinates:
  * eddy_adv                    (eddy_adv) int64 0 1 2 3 4 5 ... 13 14 15 16 17
Dimensions without coordinates: temp_adv
Data variables:
    temp_adv_ml_bin_left        (temp_adv) float64 -562.2 -537.3 ... 412.1 437.1
    temp_rate_ml_mean           (temp_adv) float64 -569.9 -545.1 ... 303.0 332.2
    temp_rate_ml_std            (temp_adv) float64 113.5 112.4 ... 131.2 132.2
    eddy_adv_ml_bin_population  (temp_adv) float64 5.098e+03 ... 5.862e+03